In [ ]:

"""
# Демонстрация мультиагентной системы генерации гипотез

## 1. Импорт библиотек
"""

import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

from src.core.orchestrator import Orchestrator
from src.config import ConfigManager

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8-whitegrid')

"""
## 2. Загрузка данных
"""

titanic_path = Path("../data/titanic.csv")

if not titanic_path.exists():
    import seaborn as sns
    df = sns.load_dataset('titanic')
    df.to_csv(titanic_path, index=False)
    print(f"Создан файл: {titanic_path}")

df = pd.read_csv(titanic_path)
print(f"Размер данных: {df.shape}")
print("\nПервые 5 строк:")
display(df.head())

print("\nИнформация о данных:")
print(df.info())

print("\nСтатистика:")
display(df.describe())

"""
## 3. Настройка системы
"""

config_manager = ConfigManager()

import os
if 'OPENAI_API_KEY' in os.environ:
    config_manager.update_llm_config(
        provider="openai",
        model="gpt-3.5-turbo",
        api_key=os.environ['OPENAI_API_KEY']
    )
else:
    print("API ключ не найден. LLM будет отключен.")
    config_manager.update_llm_config(enable=False)

orchestrator = Orchestrator(
    config=config_manager,
    verbose=True
)

"""
## 4. Запуск анализа
"""

print("Запуск анализа...")
results = orchestrator.run(str(titanic_path))

print(f"\nАнализ завершен. Обработано {len(results)} гипотез.")

"""
## 5. Визуализация результатов
"""

from src.utils.visualizer import ResultVisualizer

visualizer = ResultVisualizer()
dashboard_path = "../outputs/dashboard.html"
visualizer.create_dashboard(results, dashboard_path, title="Анализ Titanic Dataset")

from IPython.display import IFrame
IFrame(src=dashboard_path, width=1000, height=900)

"""
## 6. Анализ результатов
"""

results_df = pd.DataFrame(results)

print("Топ-5 гипотез по качеству:")
top_hypotheses = results_df.sort_values('quality_score', ascending=False).head(5)
for i, row in top_hypotheses.iterrows():
    print(f"\n{i+1}. {row['hypothesis_text']}")
    print(f"   Тип: {row['hypothesis_type']}")
    print(f"   P-value: {row['p_value']:.4f}")
    print(f"   Значимость: {'Да' if row['is_significant'] else 'Нет'}")
    print(f"   Качество: {row['quality_score']:.2f}")
    print(f"   Уверенность: {row['confidence']:.2f}")

print("\n" + "="*50)
print("СВОДНАЯ СТАТИСТИКА")
print("="*50)

total = len(results_df)
significant = results_df['is_significant'].sum()
llm_enhanced = results_df['llm_enhanced'].sum()
avg_quality = results_df['quality_score'].mean()
avg_confidence = results_df['confidence'].mean()

print(f"Всего гипотез: {total}")
print(f"Значимых гипотез: {significant} ({significant/total*100:.1f}%)")
print(f"Усилено LLM: {llm_enhanced} ({llm_enhanced/total*100:.1f}%)")
print(f"Среднее качество: {avg_quality:.2f}")
print(f"Средняя уверенность: {avg_confidence:.2f}")

print("\n" + "="*50)
print("РАСПРЕДЕЛЕНИЕ ПО ТИПАМ")
print("="*50)

type_stats = results_df.groupby('hypothesis_type').agg({
    'hypothesis_id': 'count',
    'is_significant': 'sum',
    'quality_score': 'mean',
    'confidence': 'mean'
}).round(3)

type_stats.columns = ['Количество', 'Значимых', 'Ср. качество', 'Ср. уверенность']
display(type_stats)

"""
## 7. Подробный отчет по одной гипотезе
"""

best_hypothesis = results_df.loc[results_df['quality_score'].idxmax()]

print("ЛУЧШАЯ ГИПОТЕЗА:")
print("="*60)
print(f"Текст: {best_hypothesis['hypothesis_text']}")
print(f"Тип: {best_hypothesis['hypothesis_type']}")
print(f"Метод: {best_hypothesis['method']}")
print(f"P-value: {best_hypothesis['p_value']:.4f}")
print(f"Значимость: {'Да' if best_hypothesis['is_significant'] else 'Нет'}")
print(f"Качество: {best_hypothesis['quality_score']:.2f}")
print(f"Уверенность: {best_hypothesis['confidence']:.2f}")
print(f"Заключение: {best_hypothesis['conclusion']}")
print(f"Объяснение: {best_hypothesis['explanation']}")

report_path = f"../outputs/hypothesis_{best_hypothesis['hypothesis_id']}.html"
visualizer.create_hypothesis_report(best_hypothesis.to_dict(), report_path)

print(f"\nДетальный отчет сохранен: {report_path}")

"""
## 8. Визуализация данных для лучшей гипотезы
"""

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

hypothesis_type = best_hypothesis['hypothesis_type']
columns = best_hypothesis['columns_involved']

if hypothesis_type == 'mean_difference' and len(columns) >= 2:
    num_col, cat_col = columns[0], columns[1]

    sns.boxplot(x=cat_col, y=num_col, data=df, ax=axes[0])
    axes[0].set_title(f'Распределение {num_col} по {cat_col}')
    axes[0].set_xlabel(cat_col)
    axes[0].set_ylabel(num_col)

    means = df.groupby(cat_col)[num_col].mean()
    axes[1].bar(means.index.astype(str), means.values)
    axes[1].set_title(f'Средние значения {num_col} по {cat_col}')
    axes[1].set_xlabel(cat_col)
    axes[1].set_ylabel(f'Среднее {num_col}')

elif hypothesis_type == 'correlation' and len(columns) >= 2:
    col1, col2 = columns[0], columns[1]

    sns.scatterplot(x=col1, y=col2, data=df, ax=axes[0])
    axes[0].set_title(f'Корреляция между {col1} и {col2}')

    df[col1].hist(ax=axes[1], alpha=0.7, label=col1)
    df[col2].hist(ax=axes[1], alpha=0.7, label=col2)
    axes[1].set_title('Распределения переменных')
    axes[1].legend()

plt.tight_layout()
plt.show()

"""
## 9. Экспорт результатов
"""

output_dir = Path("../outputs")
output_dir.mkdir(exist_ok=True)

json_path = output_dir / "full_results.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

csv_path = output_dir / "results_summary.csv"
summary_df = results_df[[
    'hypothesis_id', 'hypothesis_text', 'hypothesis_type',
    'method', 'p_value', 'is_significant', 'quality_score',
    'confidence', 'conclusion'
]]
summary_df.to_csv(csv_path, index=False, encoding='utf-8')

print(f"\nРезультаты сохранены:")
print(f"  - JSON: {json_path}")
print(f"  - CSV: {csv_path}")
print(f"  - Дашборд: {dashboard_path}")
print(f"  - Отчет по гипотезе: {report_path}")

"""
## 10. Сравнение с ручным анализом
"""

print("РУЧНОЙ АНАЛИЗ ДЛЯ СРАВНЕНИЯ")
print("="*50)

from scipy import stats

col1, col2 = 'age', 'fare'
data = df[[col1, col2]].dropna()

corr, p_value = stats.pearsonr(data[col1], data[col2])

print(f"\nРучная проверка гипотезы:")
print(f"Гипотеза: Существует корреляция между {col1} и {col2}")
print(f"Метод: Корреляция Пирсона")
print(f"Результат: r = {corr:.3f}, p = {p_value:.4f}")
print(f"Заключение: {'Есть значимая корреляция' if p_value < 0.05 else 'Нет значимой корреляции'}")

auto_result = None
for r in results:
    if col1 in r['hypothesis_text'] and col2 in r['hypothesis_text']:
        auto_result = r
        break

if auto_result:
    print(f"\nСРАВНЕНИЕ С АВТОМАТИЧЕСКИМ АНАЛИЗОМ:")
    print(f"Ручной p-value: {p_value:.4f}")
    print(f"Автоматический p-value: {auto_result['p_value']:.4f}")
    print(f"Разница: {abs(p_value - auto_result['p_value']):.4f}")